# Project: Momentum

## Setting up the Environment

In [115]:
import datetime as dtm    
from time import time
import warnings
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
import math

import sigtech.framework.infra.cal as cal
import sigtech.framework as sig
from sigtech.framework.services.strategy_service.service import *
from sigtech.framework.strategies.reinvestment_strategy import get_single_stock_strategy
from sigtech.framework.analytics.performance.metrics import summary
from scipy import stats
from scipy.stats.mstats import winsorize

import QILibrary as q

In [116]:
env = q.config.init_environment()

object_service = env.object_service()
strategy_cache = env.strategy_service()

## Construct our trading universe of stocks

In [ ]:
# fix a start and end date for the backtests.
start_date = dtm.datetime(2002, 2, 1)
end_date = dtm.datetime(2024, 2, 29)

# define universe name
universe_name = 'SPX IB UNIVERSE 2002-2024'

# check the status of the universe and create the universe if it doesn't exist. 
# monthly data
universe = q.universe_setup.universe_check(universe_name, start_date, end_date)
universe

In [ ]:
HISTORY_START_DATE = dtm.datetime(2000, 1, 3)
strategy_name = "IB REINVEST"
universe, rs = q.reinvestment_securities.reinvestment_security_check(universe, HISTORY_START_DATE, strategy_name, universe_name)

In [ ]:
unique_dates = universe.index.get_level_values('Date').unique()
print(unique_dates)  # monthly data

In [ ]:
universe

### Downloading the stock characteristics

In [ ]:
signal_list = ['Price12Mom',  
               'Price6Mom', 
               'EarningGrowth',
               'Value_BY',
               'Size', 
               'Value_EY',
               'Monthly3Vol',
               'Quality',
               'ROE',
               'InvToAssets',
               
               'Beta',
               'Profitability',
               'CapExToAssets',
               'Value_CFY',
               
               'Price6Mom1',
               'Price3Mom',
               'PB_ratio',
               'PE_ratio',
               'PS_ratio',
               'div_yield',
               'DebtToEquity',
               'operating_margin'
               ]

recalculate_indices = []

universe = q.universe_setup.add_signals_main(signal_list, universe_name,
                                               recalculate_indices=recalculate_indices)
universe

In [ ]:
universe.columns

### Building Portfolio of each Factor

In [ ]:
universe['QualityAgg'] = universe[['Quality', 'ROE','Profitability']].groupby('Date').apply(q.signals.create_aggregate_score, limits = [0.05,0.05]).droplevel(level = 0)
universe['Value'] = universe[['Value_BY', 'Value_EY', 'Value_CFY']].groupby('Date').apply(q.signals.create_aggregate_score, limits = [0.05,0.05]).droplevel(level = 0)
# universe['Momentum'] = universe[['Price6Mom', 'EarningGrowth']].groupby('Date').apply(q.signals.create_aggregate_score, limits = [0.05,0.05]).droplevel(level = 0)
universe['ARP'] = universe[['Quality', 'Value']].groupby('Date').apply(q.signals.create_aggregate_score, limits = [0.0,0.0]).droplevel(level = 0)

universe

In [ ]:
n = 5
factor_list = ['Price12Mom','Price6Mom','Price3Mom','Size','Monthly3Vol','InvToAssets', 'Beta', 'Profitability',
       'CapExToAssets','UpDown','PE_ratio','PB_ratio','PS_ratio', 'div_yield', 'DebtToEquity',
       'operating_margin','Quality','QualityAgg','Value','ARP']
portfolio_type = 'Long_Short'
universe = q.portfolio.create_portfolio(universe, factor_list, n, portfolio_type = portfolio_type)

In [ ]:
universe.columns

In [ ]:
HISTORY_START_DATE = dtm.datetime(2000, 1, 3)
strategy_name = "IB REINVEST"
universe, rs = q.reinvestment_securities.reinvestment_security_check(universe, HISTORY_START_DATE, strategy_name, universe_name)

In [ ]:
port_names = ['Price12Mom','Price6Mom','Price3Mom','Size','Monthly3Vol','InvToAssets', 'Beta', 'Profitability',
       'CapExToAssets','UpDown','PE_ratio','PB_ratio', 'PS_ratio','div_yield', 'DebtToEquity',
       'operating_margin','Quality','QualityAgg','Value','ARP']
strategies, strategy_histories = q.signal_strategies.signal_strategies(universe, port_names=port_names, include_trading_costs=False)

In [ ]:
strategy_names = [s.split(' ')[0] for s in [item[2] for item in list(strategy_histories.columns.values)]]
matches = [j for i, item1 in enumerate(strategy_names) for j, item2 in enumerate(port_names) if item1.lower() == item2.lower()]
strategy_names = [port_names[i] for i in matches]
strategy_histories.columns = strategy_names

strategy_histories

#### Factor Momentum (Combined)

In [129]:
# get the rebalance dates (monthly)
start_date = dtm.datetime(2002, 8, 29)
end_date = dtm.datetime(2024, 2, 29)

rebalance_dates = sig.SchedulePeriodic(start_date=start_date, end_date=end_date,
        holidays='NYSE(T) CALENDAR', frequency='EOM', bdc=cal.BDC_FOLLOWING).all_data_dates()

In [130]:
# use it later: every day return
return_day = strategy_histories.pct_change().stack().rename_axis(['Date', 'internal_id']).to_frame(name='factormom')
return_day = return_day.reset_index(level=1, drop=False)
# return_day

In [131]:
# long: choose the top n factors
def top_n(group, n=4):
    # Sort the group by the factormom column in descending order
    sorted_group = group.sort_values('factormom', ascending=False)
    
    # Keep the top N values
    top_n_values = sorted_group.head(n)
    top_n_values = top_n_values[top_n_values['factormom'] > 0] # combine ts

    return top_n_values

In [132]:
# short: choose the bottom n factors
def bottom_n(group, n=4):
    # Sort the group by the factormom column in descending order
    sorted_group = group.sort_values('factormom', ascending=False)
    
    # Keep the top N values
    bottom_n_values = sorted_group.tail(n)
    bottom_n_values = bottom_n_values[bottom_n_values['factormom'] <= 0] # combine ts
    
    return bottom_n_values

In [133]:
def rebalancing(factormom_sorted, n=4):
    factormom_rebalanced = factormom_sorted.reset_index()
    
    # Loop through all the rows
    for i in range(n, len(factormom_rebalanced)):
        if factormom_rebalanced.loc[i, 'Date'] not in rebalance_dates:
            previous_row_index = i - n
            previous_row_value = factormom_rebalanced.loc[previous_row_index, 'internal_id']
            factormom_rebalanced.at[i, 'internal_id'] = previous_row_value

    # Set the 'Date' column as the index again
    factormom_rebalanced = factormom_rebalanced.set_index('Date')
    factormom_rebalanced.index = factormom_rebalanced.index + pd.tseries.offsets.BusinessDay()
    return factormom_rebalanced

In [134]:
def mom_cs_combined(strategy_histories, month, long = True, n = 5):
    num = n
    #print(num)
    factormom = strategy_histories.shift(21).pct_change(month*21)
    factormom_reshaped = factormom.stack().rename_axis(['Date', 'internal_id']).to_frame(name='factormom')
    factormom_reshaped = factormom_reshaped.reset_index(level=1, drop=False)
    
    if long: 
        factormom_sorted = factormom_reshaped.groupby('Date').apply(top_n, num).reset_index(level=0, drop=True).drop('factormom', axis=1)
    else:
        factormom_sorted = factormom_reshaped.groupby('Date').apply(bottom_n, num).reset_index(level=0, drop=True).drop('factormom', axis=1)
    
    # Reset the index to convert it to a regular column
    factormom_rebalanced = rebalancing(factormom_sorted, n = num)
    return factormom_rebalanced
    

In [ ]:
# match the top n factors we choose to its return at time t
factormom_long = mom_cs_combined(strategy_histories, month=3, long = False, n = 8)
factormom_long.head(20)

In [136]:
#fac_long = pd.merge(factormom_long, return_day, on=['Date', 'internal_id'], how='inner')

In [137]:
def facmom_gross_longshort_combined(strategy_histories, month=6, port=True, n = 5):
    mon = month
    if port:
        factormom_rebalanced = mom_cs_combined(strategy_histories, month=mon, long = port, n = 5)
    else:
        factormom_rebalanced = mom_cs_combined(strategy_histories, month=mon, long = port, n = 5)
        
    fac = pd.merge(factormom_rebalanced, return_day, on=['Date', 'internal_id'], how='inner')
    facmom_return = fac.groupby('Date').mean()
    facmom_gross = facmom_return['factormom']+1
    facmom_gross = pd.DataFrame(facmom_gross.cumprod()*1000)
    
    # Get the first index value
    first_index = facmom_gross.index[0]

    # Create a new row with the desired index and 'factormom' value
    new_row = pd.DataFrame({'factormom': 1000}, index=[first_index - pd.tseries.offsets.BusinessDay()])

    # Concatenate the new row to the original DataFrame
    facmom_gross = pd.concat([new_row, facmom_gross])
    
    #f"facmom_cs_{month}_{port}" = factormom_gross
    return facmom_gross

In [ ]:
long = facmom_gross_longshort_combined(strategy_histories, month=12, port=True, n=8)
short = facmom_gross_longshort_combined(strategy_histories, month=12, port=False, n=8)
both = (long-short)/2+1000

In [ ]:
both

#### Factor Momentum (Cross-series)

In [140]:
# long: choose the top n factors
def top_n_cs(group, n=4):
    # Sort the group by the factormom column in descending order
    sorted_group = group.sort_values('factormom', ascending=False)
    
    # Keep the top N values
    top_n_values = sorted_group.head(n)
#    top_n_values = top_n_values[top_n_values['factormom'] > 0] # combine ts

    return top_n_values

In [141]:
# short: choose the bottom n factors
def bottom_n_cs(group, n=4):
    # Sort the group by the factormom column in descending order
    sorted_group = group.sort_values('factormom', ascending=False)
    
    # Keep the top N values
    bottom_n_values = sorted_group.tail(n)
#    bottom_n_values = bottom_n_values[bottom_n_values['factormom'] <= 0] # combine ts
    
    return bottom_n_values

In [142]:
def mom_cs(strategy_histories, month, long = True, n = 5):
    num = n
    factormom = strategy_histories.shift(21).pct_change(month*21)
    factormom_reshaped = factormom.stack().rename_axis(['Date', 'internal_id']).to_frame(name='factormom')
    factormom_reshaped = factormom_reshaped.reset_index(level=1, drop=False)
    
    if long: 
        factormom_sorted = factormom_reshaped.groupby('Date').apply(top_n_cs, num).reset_index(level=0, drop=True).drop('factormom', axis=1)
    else:
        factormom_sorted = factormom_reshaped.groupby('Date').apply(bottom_n_cs, num).reset_index(level=0, drop=True).drop('factormom', axis=1)
    
    # Reset the index to convert it to a regular column
    factormom_rebalanced = rebalancing(factormom_sorted, n = num)
    return factormom_rebalanced

In [ ]:
# match the top n factors we choose to its return at time t
factormom_long = mom_cs(strategy_histories, month=3, long = False, n = 8)
factormom_long.head(20)

In [144]:
def facmom_gross_longshort(strategy_histories, month=6, port=True, n = 5):
    mon = month
    if port:
        factormom_rebalanced = mom_cs(strategy_histories, month=mon, long = port, n = 5)
    else:
        factormom_rebalanced = mom_cs(strategy_histories, month=mon, long = port, n = 5)
        
    fac = pd.merge(factormom_rebalanced, return_day, on=['Date', 'internal_id'], how='inner')
    facmom_return = fac.groupby('Date').mean()
    facmom_gross = facmom_return['factormom']+1
    facmom_gross = pd.DataFrame(facmom_gross.cumprod()*1000)
    
    # Get the first index value
    first_index = facmom_gross.index[0]

    # Create a new row with the desired index and 'factormom' value
    new_row = pd.DataFrame({'factormom': 1000}, index=[first_index - pd.tseries.offsets.BusinessDay()])

    # Concatenate the new row to the original DataFrame
    facmom_gross = pd.concat([new_row, facmom_gross])
    
    #f"facmom_cs_{month}_{port}" = factormom_gross
    return facmom_gross

#### Factor Momentum (Time-series)

In [145]:
def compare_rows(column):
    return (column > 0).astype(int)

In [146]:
def mom_ts(strategy_histories, month, longshort = 1):
    factormom = strategy_histories.shift(21).pct_change(month*21).dropna()
    ts_result = factormom.apply(compare_rows)
    ts_result_reshaped = ts_result.stack().rename_axis(['Date', 'internal_id']).to_frame(name='factormom')
    ts_result_reshaped = ts_result_reshaped.reset_index(level=1, drop=False)
    ts_result_reshaped.index = ts_result_reshaped.index + pd.tseries.offsets.BusinessDay()
    fac_ts = pd.merge(ts_result_reshaped, return_day, on=['Date', 'internal_id'], how='inner')
    
    # Multiply columns and create a new column 'result'
    fac_ts = ts_return_all(fac_ts, long = longshort)

    # Delete 'factormom_x' and 'factormom_y' columns
    fac_ts.drop(['factormom_x', 'factormom_y'], axis=1, inplace=True)
    
    facmom_return = fac_ts.groupby('Date').mean()
    facmom_ts_gross = facmom_return['fac_ts']+1
    facmom_ts_gross = pd.DataFrame(facmom_ts_gross.cumprod()*1000)
    
    # Get the first index value
    first_index = facmom_ts_gross.index[0]

    # Create a new row with the desired index and 'factormom' value
    new_row = pd.DataFrame({'fac_ts': 1000}, index=[first_index - pd.tseries.offsets.BusinessDay()])

    # Concatenate the new row to the original DataFrame
    facmom_ts_gross = pd.concat([new_row, facmom_ts_gross])
    
    return facmom_ts_gross

In [147]:
def ts_return_all(fac_ts, long = 1):
    if long == 1:
        fac_ts['fac_ts'] = fac_ts['factormom_x'] * fac_ts['factormom_y']
    elif long == 0:
        fac_ts['fac_ts'] = (fac_ts['factormom_x']-1) * fac_ts['factormom_y']
    else:
        fac_ts['fac_ts'] = fac_ts['factormom_x'] * fac_ts['factormom_y']+(fac_ts['factormom_x']-1) * fac_ts['factormom_y']
    return fac_ts

In [148]:
#ts = mom_ts(strategy_histories, month=12, long = True)

In [149]:
ts = mom_ts(strategy_histories, month=12, longshort = 0)

In [ ]:
ts

In [151]:
momentum = strategy_histories.iloc[:, :3]

In [152]:
formation = [3,6,12]

In [ ]:
for mon in formation:
    long = facmom_gross_longshort(strategy_histories, month=int(mon), port=True, n=8)
    short = facmom_gross_longshort(strategy_histories, month=int(mon), port=False, n=8)
    both = (long-short)/2+1000
    
    long_cbn = facmom_gross_longshort_combined(strategy_histories, month=int(mon), port=True, n=8)
    short_cbn = facmom_gross_longshort_combined(strategy_histories, month=int(mon), port=False, n=8)
    both_cbn = (long_cbn-short_cbn)/2+1000
    
    long_ts = mom_ts(strategy_histories, month=int(mon), longshort = 1)
    short_ts = mom_ts(strategy_histories, month=int(mon), longshort = 0)
    both_ts = mom_ts(strategy_histories, month=int(mon), longshort = 3)
#    ts = mom_ts(strategy_histories, month=mon, long = 1)
    
    momentum[f"Facmom_CS_{mon}"] = both['factormom']
    #momentum[f"Facmom_CS_{mon}_Short"] = short['factormom']
    momentum[f"Facmom_Combined_{mon}"] = both_cbn['factormom']
    momentum[f"Facmom_TS_{mon}"] = both_ts['fac_ts']

In [ ]:
momentum

In [155]:
# strategy_histories['FacMom'] = facmom_gross['factormom']
# strategy_histories['FacMom_ts'] = facmom_ts_gross['fac_ts']

In [156]:
start = dtm.datetime(2018, 2, 1)
end = dtm.datetime(2024, 2, 29)
# factor_names_hist = ['Value', 'Price12Mom', 'Price6Mom','Price3Mom','Size', 'ARP', 'Quality', 'Facmom_Combined_3', 'Facmom_Combined_6', 'Facmom_Combined_12']
# factor_names_hist = ['Price12Mom', 'Price6Mom','Price3Mom', 'Facmom_CS_3', 'Facmom_CS_6', 'Facmom_CS_12', 'Facmom_TS_3','Facmom_TS_6','Facmom_TS_12']
factor_names_hist = ['Price12Mom', 'Price6Mom','Price3Mom', 'Facmom_TS_3','Facmom_TS_6','Facmom_TS_12', 'Facmom_Combined_3', 'Facmom_Combined_6', 'Facmom_Combined_12', 'Facmom_CS_3', 'Facmom_CS_6', 'Facmom_CS_12']

In [157]:
def plot_history(history, start = start, end = end, factornames = factor_names_hist):
    benchmark = history.loc[start]
    history = history.divide(benchmark)*1000
    history = history.loc[start:end]
    history[factornames].plot(figsize=(20, 10))
    return history

In [ ]:
#plot_history(history = strategy_histories)
plot_history(history = momentum, start = dtm.datetime(2018, 2, 1), end = dtm.datetime(2024, 2, 29), factornames = factor_names_hist)

In [ ]:
# factor_names_hist = ['Value', 'Price12Mom', 'Price6Mom','Price3Mom','Size', 'ARP', 'Quality']
factor_names_hist = ['Price12Mom', 'Price6Mom','Price3Mom', 'Facmom_CS_3', 'Facmom_CS_6', 'Facmom_CS_12', 'Facmom_TS_3','Facmom_TS_6','Facmom_TS_12']
plot_history(history = momentum, start=dtm.datetime(2003, 3, 5), end=dtm.datetime(2024, 2, 29), factornames = factor_names_hist)

### Compare Momentum with Others

In [160]:
strategy_all = strategy_histories.iloc[:, 3:].join(momentum, how='inner')

In [161]:
selected_strat = strategy_all[['Size','UpDown','Quality','Value','ARP','Price6Mom', 'Price3Mom', 'Facmom_TS_6', 'Facmom_CS_6']]

In [ ]:
strategy_all.columns

In [ ]:
# factor_names_hist = ['Value', 'Price12Mom', 'Price6Mom','Price3Mom','Size', 'ARP', 'Quality']
factor_names_hist = ['Size','UpDown','Quality','Value','ARP','Price6Mom', 'Price3Mom', 'Facmom_TS_6', 'Facmom_CS_6']
plot_history(history = selected_strat, start = dtm.datetime(2018, 2, 1), end = dtm.datetime(2024, 2, 29), factornames = factor_names_hist)

In [ ]:
plot_history(history = selected_strat, start = dtm.datetime(2004, 1, 29), end = dtm.datetime(2024, 2, 29), factornames = factor_names_hist)

### Table Summary

In [ ]:
from sigtech.framework.analytics.performance.metrics import summary
 
res = summary(selected_strat[dtm.datetime(2018, 2, 1):], cash=sig.CashIndex.from_currency("USD").history())

#res = summary(strategy_histories, cash=sig.CashIndex.from_currency("USD").history())
res.iloc[0:8,:][factor_names_hist]

In [ ]:
strategy_histories

In [ ]:
few_views = [sig.View.SUMMARY_SINGLE, sig.View.DRAWDOWN_PLOT, sig.View.ROLLING_PLOTS ]
sig.PerformanceReport(selected_strat.iloc[:,[5,6,7,8]], cash=(sig.CashIndex.from_currency("USD").history()*0+1), views=few_views).report()

### Correlation

In [ ]:
strategy_all = momentum.sort_index()
strategy_returns = momentum.pct_change().dropna()

R = np.corrcoef(strategy_returns.fillna(0), rowvar=False)
R = pd.DataFrame(R, columns = strategy_returns.columns, index = strategy_returns.columns)
R

In [ ]:
sta = strategy_histories.shift().pct_change(21)
sta

In [171]:
import numpy as np
import statsmodels

In [172]:
sta = sta.dropna().abs()

In [ ]:
sta

In [ ]:
for column in sta.columns:
    x = sta[column].values

    # Estimate AR(1) coefficient
    model = statsmodels.tsa.ar_model.AutoReg(x, lags=1)
    result = model.fit()
    ar_coefficient = result.params[1]
    
    p_value = result.pvalues[1]
    is_significant = p_value < 0.05

    print(f"AR(1) coefficient for column {column}: {ar_coefficient}")
    print(f"Is significant for column {column}: {is_significant}")

### Spanning test

In [176]:
import pandas as pd
import statsmodels.api as sm

In [180]:
selected_strat = selected_strat['2018-02-01':'2024-02-29']

In [ ]:
selected_strat

In [183]:
selected_strat = selected_strat.replace([float('inf'), float('-inf')], pd.NA).dropna()

In [ ]:
results = {}

# Loop through each pair of columns
for i in range(len(selected_strat.columns)):
    for j in range(len(selected_strat.columns)):
        if i != j:
            X = selected_strat.iloc[:, j]  # Independent variable
            y = selected_strat.iloc[:, i]   # Dependent variable
            
            # Add a constant term for the intercept
            X = sm.add_constant(X)
            
            # Fit regression model
            model = sm.OLS(y, X).fit()
            results[(selected_strat.columns[i], selected_strat.columns[j])] = model.summary()

# Display the results
for pair, summary in results.items():
    print(f'Regression result for {pair}:')
    print(summary)